Recall, we need to sample $\vec{\gamma} \in \mathbb{R}^k$ and $\vec{\eta}\in\mathbb{R}^k$ such that $\sum_{i=1}^k\sqrt{p_i}\gamma_i=0$, $\sum_{i=1}^k\gamma_i^2=\varphi^2$ and $\sum_{i=1}^k\eta_i^2=1-\varphi^2$.

First, we build an orthonormal basis of the hyperplane orthogonal to $(\sqrt{p_1},\ldots,\sqrt{p_k})$, $F$.

In [2]:
import numpy as np

def get_Fsj(p, s, j):
    if s == 1:
        if j == 1:
            return -np.sqrt(p[1])
        if j == 2:
            return np.sqrt(p[0])
        else:
            return 0
    else:
        if j <= s:
            return -np.sqrt(p[j-1]*p[s]) / np.sqrt(np.sum(p[0:s]))
        if j == s + 1:
            return np.sqrt(np.sum(p[0:s]))
        else:
            return 0

def get_Fs(p, s, k):
    Fs = list()
    for i in range(1, k + 1):
        vec_elem = get_Fsj(p, s, i)
        Fs.append(vec_elem)
    return np.array(Fs)

def get_orthonormal_basis(p, k):
    F = list()
    for i in range(1, k):
        orth_vec = get_Fs(p, i, k)
        F.append(orth_vec / np.linalg.norm(orth_vec))
    return np.array(F)

We can go ahead and get an orthonormal basis for $k=20$ where all weights are uniform $(\frac{1}{20})$.

In [5]:
k = 20
p = np.ones(k) * 1 / float(k)
basis = get_orthonormal_basis(p, k)

This basis, $b$, has the property that for any vector, $v\in\mathbb{R}^{k-1}$, where $d=b^T\cdot v$, $\sum_{i=1}^kd_i=0$. This means that we can sample a random vector $v$ and dot it with the transpose of the basis and end up on the hyperplane.

In [9]:
v = np.random.normal(1, 0, k - 1)
d = np.dot(basis.T, v)
print(np.isclose(np.sum(d), 0))

True


We also need this vector to be on the hypersphere with radius $\varphi$ since we need $\sum_{i=1}^k\gamma_i^2=\varphi^2$. There is a simple way to sample a point on a hypersphere centered at the origin. First, we sample a vector $\vec{q}\sim\mathcal{N}(0, 1)^k$. Then we simply multiply the vector by the radius of the hypersphere $\varphi$ divided by the magnitude of the vector. Here is an example:

In [14]:
varphi2 = np.random.beta(2, 2) # this is a randomly drawn parameter in the generative phase. I've arbitrarily chosen
# the parameters alpha=2 and beta=2. 
q = np.random.normal(0, 1, k)
q *= np.sqrt(varphi2) / np.linalg.norm(q)

print(np.isclose(np.sum(q**2), varphi2))

True


We can use a trick to sample from the hypersphere while remaining on the hyperplane. Instead of sampling a new $q$, just use $d$, which we already know to be on the hyperplane. Since putting it on the hyperplane simply involves a rescaling, we can put $d$ on the hyperplane while maintaining $\sum_{i=1}^kd_i=0$, thus giving us $\vec{\gamma}$.

In [19]:
gamma = d * np.sqrt(varphi2) / np.linalg.norm(d)

print(np.isclose(np.sum(gamma * p), 0))
print(np.isclose(np.sum(gamma**2), varphi2))

True
True


We can see these values look much more uniformly distributed compared to those sampled in the way described in the paper.

In [20]:
print(gamma)

[-0.49579783 -0.26321682 -0.17808626 -0.12246537 -0.08102013 -0.0479608
 -0.02045452  0.00310019  0.02369807  0.04199971  0.05846632  0.0734327
  0.0871496   0.09980972  0.11156434  0.12253442  0.13281817  0.14249646
  0.15163663  0.1602954 ]


Next, we need to sample $\vec{\eta}\in\mathbb{R}^k$ which has the following constraints: $\eta_i \geq 0\text{ }\forall i$ and $\sum_{i=1}^k\eta_i^2=1-\varphi^2$ a (hyper-quadrant?) with radius $\sqrt(1-\varphi^2)$. To do this, we sample a hypersphere in the way described above, however, we ensure that all vector values are positive.

In [25]:
tmp = np.random.normal(0, 1, k)
eta = np.abs(tmp * np.sqrt(1 - varphi2) / np.linalg.norm(tmp))

print(np.isclose(np.sum(eta**2), 1 - varphi2))

True


From here, we work backward to get $\vec{\mu}$ and $\vec{\sigma}$. We know $\alpha_i = \frac{\gamma_i}{\sqrt{p_i}}$, $\tau_i = \frac{\eta_i}{\sqrt{p_i}}$, $\mu_i=\mu+\sigma\gamma_i$, and $\sigma_i=\sigma\tau_i$. Let's assume the parent mean was 500 and the parent standard deviation was 120.

In [27]:
mu = 500
sigma = 120

alpha = gamma / np.sqrt(p)
tau = eta / np.sqrt(p)
mus = mu + sigma * gamma
sigmas = sigma * tau

print(mus)
print(sigmas)

[440.50425985 468.41398172 478.62964894 485.30415598 490.27758412
 494.24470372 497.5454579  500.37202267 502.84376827 505.03996546
 507.01595858 508.8119236  510.45795169 511.97716619 513.38772104
 514.70413031 515.93818071 517.09957497 518.19639596 519.23544835]
[ 67.00814577 104.45647697 105.85626201 137.8648934   38.51682559
 168.90943311  63.25182731  35.83831666  48.76221624  58.23665456
 163.20329778  44.80340111  22.38388369  36.57301155   4.23065445
  84.93790126 114.19133032  38.51015761  21.73604741  24.6642619 ]


Putting it all together...

In [32]:
import numpy as np

def get_Fsj(p, s, j):
    if s == 1:
        if j == 1:
            return -np.sqrt(p[1])
        if j == 2:
            return np.sqrt(p[0])
        else:
            return 0
    else:
        if j <= s:
            return -np.sqrt(p[j-1]*p[s]) / np.sqrt(np.sum(p[0:s]))
        if j == s + 1:
            return np.sqrt(np.sum(p[0:s]))
        else:
            return 0

def get_Fs(p, s, k):
    Fs = list()
    for i in range(1, k + 1):
        vec_elem = get_Fsj(p, s, i)
        Fs.append(vec_elem)
    return np.array(Fs)

def get_orthonormal_basis(p, k):
    F = list()
    for i in range(1, k):
        orth_vec = get_Fs(p, i, k)
        F.append(orth_vec / np.linalg.norm(orth_vec))
    return np.array(F)

def get_gamma(p, k, varphi2):
    basis = get_orthonormal_basis(p, k)
    v = np.random.normal(1, 0, k - 1)
    d = np.dot(basis.T, v)
    gamma = d * np.sqrt(varphi2) / np.linalg.norm(d)
    assert(np.isclose(np.sum(gamma * p), 0))
    assert(np.isclose(np.sum(gamma**2), varphi2))
    return gamma

def get_eta(k, varphi2):
    tmp = np.random.normal(0, 1, k)
    eta = np.abs(tmp * np.sqrt(1 - varphi2) / np.linalg.norm(tmp))
    assert(np.isclose(np.sum(eta**2), 1-varphi2))
    return eta

def get_alpha(p, k, varphi2):
    gamma = get_gamma(p, k, varphi2)
    alpha = gamma / np.sqrt(p)
    return alpha

def get_tau(p, k, varphi2):
    eta = get_eta(k, varphi2)
    tau = eta / np.sqrt(p)
    return tau

def get_mus(p, k, varphi2, mu, sigma):
    alphas = get_alpha(p, k, varphi2)
    mus = mu + sigma * alphas
    return mus

def get_sigmas(p, k, varphi2, sigma):
    tau = get_tau(p, k, varphi2)
    sigmas = sigma * tau
    return sigmas

def get_mixture(mu, sigma, k, p, varphi2):
    return (get_mus(p, k, varphi2, mu, sigma), get_sigmas(p, k, varphi2, sigma))

And now for testing...

In [34]:
mu = 450
sigma = 100
k = 25
p = np.ones(k) * 1./k
varphi2 = .2

mus, sigmas = get_mixture(mu, sigma, k, p, varphi2)
print(mus)
print(sigmas)

[302.21208284 366.76180527 390.38864349 405.82547154 417.32803709
 426.50320805 434.13720624 440.67449715 446.39116079 451.47053493
 456.04062013 460.1943356  464.0012763  467.51492201 470.77725923
 473.82185612 476.67597309 479.36205065 481.89878285 484.3019076
 486.58479888 488.75891778 490.83416074 492.81913208 494.72135955]
[ 90.46999159  21.20775135  63.92117371 219.54387536  89.17077058
  54.68429205 116.03002534  15.92487433  39.19070503 103.81095555
 143.31079937  21.9098818   28.23346198 130.632348    56.75973026
 101.13569624  13.82446577  23.50862883  63.02442855  64.09250542
 149.30316985  75.55268655   5.21925044  68.65025949  90.639307  ]


We can see that the value of $\varphi^2$ can drastically change the mixture:

In [35]:
varphi2 = .999
mus, sigmas = get_mixture(mu, sigma, k, p, varphi2)
print(mus)
print(sigmas)

[119.70144421 263.96682468 316.77161881 351.27215246 376.97980748
 397.48585272 414.54745442 429.15797054 441.93442601 453.28657154
 463.50048192 472.78382692 481.29214777 489.14496906 496.43612853
 503.24064933 509.6194571  515.62270522 521.29217401 526.66303686
 531.76518397 536.62423026 541.26229383 545.69860485 549.94998749]
[0.24696756 4.26219185 1.54058542 0.78088689 2.21331787 3.29251986
 0.60959526 0.3700824  2.89867775 4.69069734 4.74977946 1.971672
 2.70168157 4.45313683 2.15344154 8.02568045 1.10661989 1.22195593
 1.19744301 2.4096889  1.65475534 4.99137384 1.51987003 3.74942036
 2.35198456]


In [36]:
varphi2 = .001
mus, sigmas = get_mixture(mu, sigma, k, p, varphi2)
print(mus)
print(sigmas)

[439.54981616 444.11417081 445.78484056 446.87638914 447.68974335
 448.33852591 448.8783311  449.34058737 449.74481653 450.10398252
 450.42713635 450.72084838 450.99003974 451.23849201 451.46917409
 451.6844596  451.88627615 452.07621051 452.25558457 452.42551115
 452.58693594 452.74066936 452.8874112  453.02776987 453.16227766]
[5.68489972e+01 8.70524439e+01 3.90674335e+01 5.09015744e+01
 7.06782387e+01 1.61640578e+02 5.08909664e+01 7.41585578e+01
 2.09424877e+02 6.85382482e+01 2.25298171e+01 1.16553179e+01
 1.13493980e+02 7.81659484e+01 1.78182434e+01 1.03347136e+02
 1.09906590e+02 9.17566198e+01 2.23329336e+02 2.12512094e-01
 9.59778610e-01 1.63179156e+02 1.20971691e+02 6.76577735e+01
 1.43830611e+01]
